# Reference portfolios

#### Step 1: Upload instruments

```sql
-- ============================================================
-- Description:
-- 1. In this query, we run an ETL process on some instruments.
-- 2. First, we load an Excel file of instruments from Drive.
-- 3. Next, we transform the shape of the instrument data.
-- 4. Finally we upload the instrument data into LUSID.
-- ============================================================

-- Extract equity instrument data from LUSID Drive

@instruments_data = use Drive.Excel
--file=/luminesce-examples/reference_port.xlsx
--worksheet=instruments
enduse;

-- Transform data using SQL

@equity_instruments =
select
Name as DisplayName,
ISIN as Isin,
ClientInternal as ClientInternal,
SEDOL as Sedol,
'GBP' as DomCcy
from @instruments_data;

-- Upload the transformed data into LUSID

select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @equity_instruments;
```

#### Step 2: Create reference portfolio

```sql
-- ============================================================
-- Description:
-- In this query, we create 2 reference portfolios.
-- ============================================================

-- Extract portfolio constituent data from LUSID Drive

@portfolio_data = use Drive.Excel
--file=/luminesce-examples/reference_port.xlsx
--worksheet=constituents
enduse;

-- Define some variables for portfolio creation

@@scope = select 'luminesce-examples';
@@iso_currency = select 'GBP';
@portfolio_codes = select distinct port as [port_code], currency from @portfolio_data;

-- Define the upsert reference portfolio request

@create_portfolio_request = select 'Reference' as PortfolioType,
@@scope as PortfolioScope,
port_code as PortfolioCode,
port_code as DisplayName,
port_code as Description,
#2000-01-01# as Created,
currency  as BaseCurrency from @portfolio_codes;

-- Upload the portfolio into LUSID

select * from Lusid.Portfolio.Writer
where ToWrite = @create_portfolio_request;

```

#### Step 3: Upload constituents

```sql
-- ============================================================
-- Description:
-- In this query, we then add some constituents with different
-- weights to the two new reference portfolios
-- ============================================================

-- Extract portfolio constituent data from LUSID Drive

@portfolio_data = use Drive.Excel
--file=/luminesce-examples/reference_port.xlsx
--worksheet=constituents
enduse;

@@scope = select 'luminesce-examples';

-- Transform extracted data

@constituents =
select
currency as Currency,
effective_date as EffectiveFrom,
port as PortfolioCode,
@@scope as PortfolioScope,
inst_id as ClientInternal,
weight as Weight
from @portfolio_data;

-- Add constituents to reference portfolios

select * from Lusid.Portfolio.Constituent.Writer
where ToWrite = @constituents;

```